In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

# Positional Encoding

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length=5000):
        super(PositionalEncoding, self).__init__()
        
        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * 
                           (-math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        self.register_buffer('pe', pe.unsqueeze(0))
        
    def forward(self, x):
        # pe is automatically on the correct device due to register_buffer
        return x + self.pe[:, :x.size(1)]


# Multihead Attention

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0
        
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        attn_probs = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, V)
        return output
    def forward(self, query, key, value, mask=None):
        batch_size = query.size(0)
        
        Q = self.W_q(query)
        K = self.W_k(key)
        V = self.W_v(value)
        
        Q = Q.view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        K = K.view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        V = V.view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        
        attn_output = attn_output.transpose(1, 2).contiguous().view(
            batch_size, -1, self.d_model)
        
        output = self.W_o(attn_output)
        return output


In [ ]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))


# Encoder Layer

In [ ]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x


# Decoder Layer

In [ ]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x

# Transformer

In [ ]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, 
                 num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)
        
        self.encoder_layers = nn.ModuleList([
            EncoderLayer(d_model, num_heads, d_ff, dropout) 
            for _ in range(num_layers)
        ])
        self.decoder_layers = nn.ModuleList([
            DecoderLayer(d_model, num_heads, d_ff, dropout) 
            for _ in range(num_layers)
        ])
        
        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src, tgt):
        # Get device from input tensors
        device = src.device
        
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        
        # Create nopeak_mask on the same device as input tensors
        nopeak_mask = (1 - torch.triu(
            torch.ones(1, seq_length, seq_length, device=device), 
            diagonal=1
        )).bool()
        
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        
        src_embedded = self.dropout(self.positional_encoding(
            self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(
            self.decoder_embedding(tgt)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)
        return output


# Handling Data

In [ ]:
import pandas as pd
import os

# Check available files
# print(os.listdir('/kaggle/input/hindi-english-parallel-corpus/'))

# Load your data
df = pd.read_csv('/kaggle/input/hindi-english-parallel-corpus/hindi_english_parallel.csv')
df.head()

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import re

class TranslationCSVDataset(Dataset):
    def __init__(self, csv_file, src_col, tgt_col, src_tokenizer, tgt_tokenizer, 
                 max_length=512, test_size=None):
        """
        Args:
            csv_file (string): Path to the csv file with translation data
            src_col (string): Column name for source language
            tgt_col (string): Column name for target language
            src_tokenizer: Tokenizer for source language
            tgt_tokenizer: Tokenizer for target language
            max_length (int): Maximum sequence length
            test_size (float): If provided, will split data into train/test
        """
        # Load CSV data
        self.df = pd.read_csv(csv_file)
        
        # Clean and filter data
        self.df = self.df.dropna(subset=[src_col, tgt_col])
        self.df = self.df[self.df[src_col].str.len() > 0]
        self.df = self.df[self.df[tgt_col].str.len() > 0]
        
        # Split data if test_size is provided
        if test_size:
            self.train_df, self.test_df = train_test_split(
                self.df, test_size=test_size, random_state=42
            )
            self.df = self.train_df  # Use training data by default
        
        self.src_col = src_col
        self.tgt_col = tgt_col
        self.src_tokenizer = src_tokenizer
        self.tgt_tokenizer = tgt_tokenizer
        self.max_length = max_length
        
        print(f"Loaded {len(self.df)} translation pairs")
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        src_text = str(row[self.src_col]).strip()
        tgt_text = str(row[self.tgt_col]).strip()
        
        # Clean text (optional preprocessing)
        src_text = self.clean_text(src_text)
        tgt_text = self.clean_text(tgt_text)
        
        # Tokenize
        src_tokens = self.src_tokenizer.encode(src_text)[:self.max_length-2]
        tgt_tokens = self.tgt_tokenizer.encode(tgt_text)[:self.max_length-2]
        
        # Add special tokens (1=BOS, 2=EOS, 0=PAD)
        src_tokens = [1] + src_tokens + [2]
        tgt_tokens = [1] + tgt_tokens + [2]
        
        # Pad sequences
        src_tokens += [0] * (self.max_length - len(src_tokens))
        tgt_tokens += [0] * (self.max_length - len(tgt_tokens))
        
        return {
            'src': torch.tensor(src_tokens, dtype=torch.long),
            'tgt': torch.tensor(tgt_tokens, dtype=torch.long),
            'src_text': src_text,
            'tgt_text': tgt_text
        }
    
    def clean_text(self, text):
        """Basic text cleaning"""
        # Remove extra whitespace
        text = re.sub(r'\s+', ' ', text)
        # Remove special characters if needed (adjust based on your data)
        # text = re.sub(r'[^\w\s.,!?]', '', text)
        return text.strip()
    
    def get_test_data(self):
        """Return test dataset if split was performed"""
        if hasattr(self, 'test_df'):
            test_dataset = TranslationCSVDataset.__new__(TranslationCSVDataset)
            test_dataset.df = self.test_df
            test_dataset.src_col = self.src_col
            test_dataset.tgt_col = self.tgt_col
            test_dataset.src_tokenizer = self.src_tokenizer
            test_dataset.tgt_tokenizer = self.tgt_tokenizer
            test_dataset.max_length = self.max_length
            return test_dataset
        return None


In [ ]:
import pickle
from collections import Counter

class SimpleTokenizer:
    def __init__(self, vocab_size=10000):
        self.vocab_size = vocab_size
        self.word2idx = {'<PAD>': 0, '<BOS>': 1, '<EOS>': 2, '<UNK>': 3}
        self.idx2word = {0: '<PAD>', 1: '<BOS>', 2: '<EOS>', 3: '<UNK>'}
        self.vocab_built = False
    
    def build_vocab(self, texts):
        """Build vocabulary from list of texts"""
        word_counts = Counter()
        
        for text in texts:
            words = text.lower().split()
            word_counts.update(words)
        
        # Add most common words to vocabulary
        most_common = word_counts.most_common(self.vocab_size - 4)  # -4 for special tokens
        
        for i, (word, _) in enumerate(most_common):
            if word not in self.word2idx:
                idx = len(self.word2idx)
                self.word2idx[word] = idx
                self.idx2word[idx] = word
        
        self.vocab_built = True
        print(f"Built vocabulary with {len(self.word2idx)} words")
    
    def encode(self, text):
        """Convert text to list of token IDs"""
        if not self.vocab_built:
            raise ValueError("Vocabulary not built. Call build_vocab() first.")
        
        words = text.lower().split()
        return [self.word2idx.get(word, 3) for word in words]  # 3 is <UNK>
    
    def decode(self, token_ids):
        """Convert list of token IDs back to text"""
        words = [self.idx2word.get(idx, '<UNK>') for idx in token_ids 
                if idx not in [0, 1, 2]]  # Skip PAD, BOS, EOS
        return ' '.join(words)
    
    def save(self, filepath):
        """Save tokenizer to file"""
        with open(filepath, 'wb') as f:
            pickle.dump({
                'word2idx': self.word2idx,
                'idx2word': self.idx2word,
                'vocab_size': self.vocab_size
            }, f)
    
    def load(self, filepath):
        """Load tokenizer from file"""
        with open(filepath, 'rb') as f:
            data = pickle.load(f)
            self.word2idx = data['word2idx']
            self.idx2word = data['idx2word']
            self.vocab_size = data['vocab_size']
            self.vocab_built = True


In [ ]:
def create_translation_dataloaders(csv_file, src_col='source', tgt_col='target',
                                 batch_size=32, max_length=128, test_size=0.1):
    """
    Create DataLoaders for translation training from CSV file
    """
    
    # Step 1: Load data and build tokenizers
    print("Loading data and building vocabularies...")
    df = pd.read_csv(csv_file)
    df = df.dropna(subset=[src_col, tgt_col])
    
    # Extract texts for vocabulary building
    src_texts = df[src_col].tolist()
    tgt_texts = df[tgt_col].tolist()
    
    # Build tokenizers
    src_tokenizer = SimpleTokenizer(vocab_size=10000)
    tgt_tokenizer = SimpleTokenizer(vocab_size=10000)
    
    src_tokenizer.build_vocab(src_texts)
    tgt_tokenizer.build_vocab(tgt_texts)
    
    # Save tokenizers
    src_tokenizer.save('src_tokenizer.pkl')
    tgt_tokenizer.save('tgt_tokenizer.pkl')
    
    # Step 2: Create datasets
    dataset = TranslationCSVDataset(
        csv_file=csv_file,
        src_col=src_col,
        tgt_col=tgt_col,
        src_tokenizer=src_tokenizer,
        tgt_tokenizer=tgt_tokenizer,
        max_length=max_length,
        test_size=test_size
    )
    
    # Step 3: Create DataLoaders
    train_loader = DataLoader(
        dataset, 
        batch_size=batch_size, 
        shuffle=True,
        num_workers=2,
        pin_memory=True
    )
    
    # Create test loader if test data exists
    test_dataset = dataset.get_test_data()
    test_loader = None
    if test_dataset:
        test_loader = DataLoader(
            test_dataset,
            batch_size=batch_size,
            shuffle=False,
            num_workers=2,
            pin_memory=True
        )
    
    return train_loader, test_loader, src_tokenizer, tgt_tokenizer

# Custom collate function for batching
def collate_fn(batch):
    """Custom collate function to handle batching"""
    src_batch = torch.stack([item['src'] for item in batch])
    tgt_batch = torch.stack([item['tgt'] for item in batch])
    
    return {
        'src': src_batch,
        'tgt': tgt_batch,
        'src_texts': [item['src_text'] for item in batch],
        'tgt_texts': [item['tgt_text'] for item in batch]
    }


In [ ]:
# Load your CSV data
csv_file = '/kaggle/input/hindi-english-parallel-corpus/hindi_english_parallel.csv'  # Replace with your file path

# Create data loaders
train_loader, test_loader, src_tokenizer, tgt_tokenizer = create_translation_dataloaders(
    csv_file=csv_file,
    src_col='english',  # Replace with your source column name
    tgt_col='hindi',  # Replace with your target column name
    batch_size=64,
    max_length=128,
    test_size=0.1
)

# Model parameters (using tokenizer vocab sizes)
src_vocab_size = len(src_tokenizer.word2idx)
tgt_vocab_size = len(tgt_tokenizer.word2idx)
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 128
dropout = 0.1

# Initialize your transformer model (from previous conversation)
model = Transformer(
    src_vocab_size=src_vocab_size,
    tgt_vocab_size=tgt_vocab_size,
    d_model=d_model,
    num_heads=num_heads,
    num_layers=num_layers,
    d_ff=d_ff,
    max_seq_length=max_seq_length,
    dropout=dropout
)

# Training loop adaptation
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

# Training
model.train()
for epoch in range(10):
    total_loss = 0
    for batch_idx, batch in enumerate(train_loader):
        src_data = batch['src'].to(device)
        tgt_data = batch['tgt'].to(device)
        
        optimizer.zero_grad()
        
        # Forward pass
        output = model(src_data, tgt_data[:, :-1])
        
        # Calculate loss
        loss = criterion(
            output.reshape(-1, output.size(-1)), 
            tgt_data[:, 1:].reshape(-1)
        )
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        total_loss += loss.item()
        
        if batch_idx % 100 == 0:
            print(f'Epoch {epoch}, Batch {batch_idx}, Loss: {loss.item():.4f}')
    
    print(f'Epoch {epoch} Average Loss: {total_loss/len(train_loader):.4f}')


In [ ]:
def translate(model, src_sentence, src_tokenizer, tgt_tokenizer, device, max_length=100):
    model.eval()
    
    # Tokenize source sentence
    src_tokens = [1] + src_tokenizer.encode(src_sentence) + [2]  # Add BOS/EOS
    src_tensor = torch.LongTensor(src_tokens).unsqueeze(0).to(device)
    
    # Start with BOS token
    tgt_tokens = [1]
    
    with torch.no_grad():
        for _ in range(max_length):
            tgt_tensor = torch.LongTensor(tgt_tokens).unsqueeze(0).to(device)
            
            output = model(src_tensor, tgt_tensor)
            next_token_logits = output[0, -1, :]
            next_token = torch.argmax(next_token_logits).item()
            
            tgt_tokens.append(next_token)
            
            # Stop if EOS token is generated
            if next_token == 2:  # EOS token
                break
    
    # Decode tokens to text
    translated_text = tgt_tokenizer.decode(tgt_tokens[1:-1])  # Remove BOS/EOS
    return translated_text

# Example usage
src_sentence = "Hello, how are you?"
translation = translate(model, src_sentence, src_tokenizer, tgt_tokenizer, device)
print(f"Translation: {translation}")
